In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np

from sklearn import metrics

from scipy import stats

In [2]:
#import dataset
dataset_raw = pd.read_csv("./divar_dataset/divar_posts_dataset.csv")
dataset_raw.head()

,Unnamed: 0,archive_by_user,brand,cat1,cat2,cat3,city,created_at,desc,id,image_count,mileage,platform,price,title,type,year
0,0,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Tehran,Tuesday 07PM,کلاسیک و شیک و استثنایی\nچرم مالزی\nچوب راش\nف...,54761638662241,5,NaN,mobile,3850000,ست مبلمان و نهارخوری ٩ نفره,NaN,NaN
1,1,False,NaN,for-the-home,furniture-and-home-decore,antiques-and-art,Mashhad,Tuesday 07PM,"سلام,یک عدد گلدون نخل سه طبقه ی سالم دارم با پ...",42727631379133,1,NaN,mobile,30000,گلدون مصنوعی نخل,NaN,NaN
2,2,False,NaN,vehicles,cars,heavy,Mashhad,Tuesday 07PM,سریال 43j$NUMبدون شکستگی سه حلقه لاستیک نو یک ...,63194439667302,4,NaN,mobile,-1,لودر کاتر پیلار 950,NaN,NaN
3,3,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Tehran,Tuesday 07PM,مبل راحتی هفت نفره شامل سه نفره یک عدد دونفره ...,19133025491169,4,NaN,mobile,600000,مبل راحتی هفت نفره بامیز جلو مبلی,NaN,NaN
4,4,False,NaN,personal,baby-and-toys,personal-toys,Karaj,Tuesday 08PM,شارژی کنترلی سویچ حمل تا 35 کیلو صندلی برای دو...,58998998335444,2,NaN,mobile,450000,ماشین شارژی,NaN,NaN


In [3]:
dataset = dataset_raw[["cat1","cat2","cat3","price"]]
dataset = dataset.dropna()
dataset["product"] = dataset['cat2'] + "__" + dataset['cat3']
dataset = dataset[["product","price"]]
dataset

,product,price
0,furniture-and-home-decore__sofa-armchair,3850000
1,furniture-and-home-decore__antiques-and-art,30000
2,cars__heavy,-1
3,furniture-and-home-decore__sofa-armchair,600000
4,baby-and-toys__personal-toys,450000
...,...,...
947630,building-and-garden__stove-and-heating,180000
947631,audio-video__camera-camcoders,350000
947632,cars__light,17400000
947633,mobile-tablet__mobile-tablet-accessories,40000


In [4]:
dataset = dataset[dataset['price'] != -1]
dataset

,product,price
0,furniture-and-home-decore__sofa-armchair,3850000
1,furniture-and-home-decore__antiques-and-art,30000
3,furniture-and-home-decore__sofa-armchair,600000
4,baby-and-toys__personal-toys,450000
5,furniture-and-home-decore__sofa-armchair,1000000
...,...,...
947630,building-and-garden__stove-and-heating,180000
947631,audio-video__camera-camcoders,350000
947632,cars__light,17400000
947633,mobile-tablet__mobile-tablet-accessories,40000


In [5]:
grouped = dataset.groupby("product")


In [6]:
outliers = []
for name,group in grouped:
    outliers.append(group['price'].describe())
    IQR = (group['price'].quantile(0.75) - group['price'].quantile(0.25))
    # find outliers
    higher_outlier = group['price'].quantile(0.75) + (IQR * 1.5)
    lower_outlier = group['price'].quantile(0.25) - (IQR * 1.5)
    df_filtered = group[((group["price"] > higher_outlier) | (group["price"] < lower_outlier))]
    outliers.append(df_filtered)
    
outliers

[count       4651.000000
 mean      170955.109654
 std       173930.834871
 min           20.000000
 25%        50000.000000
 50%       100000.000000
 75%       250000.000000
 max      1000000.000000
 Name: price, dtype: float64,
                      product   price
 1940    animals__accessories  650000
 12559   animals__accessories  700000
 13070   animals__accessories  700000
 22730   animals__accessories  800000
 23923   animals__accessories  650000
 ...                      ...     ...
 918169  animals__accessories  600000
 922129  animals__accessories  850000
 924152  animals__accessories  700000
 925682  animals__accessories  750000
 946638  animals__accessories  700000
 
 [204 rows x 2 columns],
 count    3.066500e+04
 mean     1.347742e+05
 std      1.456874e+05
 min      1.000000e+00
 25%      4.500000e+04
 50%      8.500000e+04
 75%      1.700000e+05
 max      1.200000e+06
 Name: price, dtype: float64,
                product    price
 669     animals__birds   400000
 720   

In [7]:
outliers = []
for name,group in grouped:
    outliers.append(group['price'].describe())
    q_low = group["price"].quantile(0.01)
    q_hi  = group["price"].quantile(0.99)

    df_filtered = group[~((group["price"] < q_hi) & (group["price"] > q_low))]
    outliers.append(df_filtered)
outliers


[count       4651.000000
 mean      170955.109654
 std       173930.834871
 min           20.000000
 25%        50000.000000
 50%       100000.000000
 75%       250000.000000
 max      1000000.000000
 Name: price, dtype: float64,
                      product   price
 6480    animals__accessories     120
 10862   animals__accessories    3000
 19941   animals__accessories    2000
 22730   animals__accessories  800000
 27833   animals__accessories    2000
 ...                      ...     ...
 882418  animals__accessories     200
 911533  animals__accessories  800000
 914439  animals__accessories  800000
 915834  animals__accessories     120
 922129  animals__accessories  850000
 
 [110 rows x 2 columns],
 count    3.066500e+04
 mean     1.347742e+05
 std      1.456874e+05
 min      1.000000e+00
 25%      4.500000e+04
 50%      8.500000e+04
 75%      1.700000e+05
 max      1.200000e+06
 Name: price, dtype: float64,
                product    price
 2004    animals__birds  1000000
 2341  

dataset['price'] = dataset['price'].astype(int).round(-3)
dataset